# Model 3: using mixed data (satellite imagery and structured data) to predict where traffic accidents will happen

Additional data used to get the LSOA for each latitude and longitude point, in order to match with the population data.

Source: https://data.london.gov.uk/dataset/mylondon

### Importing the data

In [78]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)
from scipy.spatial import cKDTree
from math import *
np.random.seed(123)

In [2]:
accidents = pd.read_csv('data/accidents/London_accidents_merged.csv')
accidents.head()

,Accident_Index,Longitude,Latitude,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,1st_Road_Class,Road_Type,Speed_limit,Junction_Detail,2nd_Road_Class,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Urban_or_Rural_Area,Hour,Two_Hour_Groupings,Time_of_Day,Was_Daylight,Was_Bad_Weather,Was_Road_Dry,log_Number_of_Casualties,log_Number_of_Vehicles,LSOA,population_per_hectare,bicycle_aadf,motorbike_aadf,car_aadf,bus_aadf,light_goods_vehicle_aadf,heavy_goods_vehicle_aadf,Road,RCat
0,201301BS70003,-0.171402,51.486361,Serious,2,1,2013-01-02,Wednesday,A,Single carriageway,30.0,T or staggered junction,Unclassified,Daylight,Fine no high winds,Dry,Urban,9,8am-10am,Morning,Yes,No,Yes,0.000000,0.693147,E01002844,110.8,1634.4,860.4,14888.0,1139.8,2297.0,352.0,A3217,PA
1,201301BS70005,-0.173356,51.495115,Slight,1,2,2013-01-04,Friday,A,Single carriageway,30.0,Crossroads,A,Daylight,Other,Dry,Urban,8,8am-10am,Morning,Yes,Yes,Yes,0.693147,0.000000,E01002821,74.6,559.6,1516.0,28505.6,1396.2,3868.6,1003.0,A4,PA
2,201301BS70006,-0.210767,51.518353,Slight,1,1,2013-01-07,Monday,B,Single carriageway,30.0,Crossroads,B,Daylight,Fine no high winds,Dry,Urban,11,10am-12pm,Office hours,Yes,No,Yes,0.000000,0.000000,E01002878,133.4,2.6,3898.2,63274.8,763.4,15253.6,3185.8,A40,PA
3,201301BS70007,-0.209675,51.516808,Slight,2,1,2013-01-10,Thursday,B,Single carriageway,30.0,Crossroads,C,Daylight,Fine no high winds,Dry,Urban,10,10am-12pm,Office hours,Yes,No,Yes,0.000000,0.693147,E01002831,179.2,2.6,3898.2,63274.8,763.4,15253.6,3185.8,A40,PA
4,201301BS70009,-0.194332,51.492922,Slight,2,1,2013-01-04,Friday,A,One way street,30.0,T or staggered junction,Unclassified,Darkness - lights lit,Fine no high winds,Dry,Urban,17,4pm-6pm,Rush hour,No,No,Yes,0.000000,0.693147,E01002851,272.3,869.2,1229.8,20478.6,897.2,4951.6,1251.4,A3220,PA


In [3]:
lsoa_latlong = pd.read_csv('data/geography/UK_LSOA_bounding_boxes.csv', usecols=['lsoa', 'Latitude', 'Longitude'])
lsoa_latlong.head()

,lsoa,Latitude,Longitude
0,E01000001,51.520269,-0.0950
1,E01000001,51.519848,-0.0967
2,E01000001,51.519030,-0.0962
3,E01000001,51.516904,-0.0981
4,E01000003,51.522376,-0.0973


In [4]:
population = pd.read_csv('data/population/Population_density.csv')
population = population.drop('Unnamed: 0', axis=1)
population.head()

,LSOA,population_per_hectare
0,E01012334,0.4
1,E01012335,12.1
2,E01012366,0.3
3,E01033481,9.3
4,E01033482,6.9


In [5]:
traffic = pd.read_csv('data/traffic/Traffic_averages.csv')
traffic.head()

,S Ref Latitude,S Ref Longitude,bicycle_aadf,motorbike_aadf,car_aadf,bus_aadf,light_goods_vehicle_aadf,heavy_goods_vehicle_aadf
0,49.915023,-6.317073,238.2,96.8,539.4,27.0,379.0,40.4
1,49.912343,-6.305686,87.4,79.4,629.0,9.0,221.8,11.0
2,49.917141,-6.306114,181.8,142.6,777.2,32.2,403.8,29.4
3,49.917810,-6.298996,61.2,54.8,342.8,6.0,251.6,34.4
4,49.918585,-6.295094,33.6,20.8,165.6,0.0,150.8,15.2


In [6]:
# Combining the motor vehicle traffic columns
to_sum = ['motorbike_aadf', 'car_aadf', 'bus_aadf', 'light_goods_vehicle_aadf', 'heavy_goods_vehicle_aadf']
traffic['motor_vehicle_aadf'] = traffic[to_sum].sum(axis=1)
traffic.drop(to_sum, axis=1, inplace=True)
traffic.head()

,S Ref Latitude,S Ref Longitude,bicycle_aadf,motor_vehicle_aadf
0,49.915023,-6.317073,238.2,1082.6
1,49.912343,-6.305686,87.4,950.2
2,49.917141,-6.306114,181.8,1385.2
3,49.917810,-6.298996,61.2,689.6
4,49.918585,-6.295094,33.6,352.4


### List of danger squares (from Sabatino)

In [7]:
model3_dataset_danger = pd.read_csv('data/accidents/model3_dataset_danger.csv')
model3_dataset_danger = model3_dataset_danger.drop('Unnamed: 0', axis=1)
model3_dataset_danger.head()

,Latitude,Longitude,population_per_hectare,bicycle_aadf,lat_4dp,long_4dp,grid_square,motor_vehicle_aadf
0,51.522443,-0.127143,55.0,782.2,51.5225,-0.1270,"51.5225,-0.127",2076.0
1,51.558820,-0.098667,158.2,294.8,51.5590,-0.0985,"51.559,-0.0985",225.8
2,51.447056,0.200494,40.5,79.4,51.4470,0.2005,"51.447,0.2005",9458.0
3,51.487995,-0.221483,96.7,1536.2,51.4880,-0.2215,"51.488,-0.2215",22292.4
4,51.622984,-0.090497,93.7,84.6,51.6230,-0.0905,"51.623,-0.0905",19072.8


### List of safe squares

In [8]:
# Getting a list of all possible locations
# Setting parameters
lat_min, lat_max = 51.257, 51.719
lon_min, lon_max = -0.542, 0.291
grid_size = 0.0005

# Getting a list of all grid squares
lats = np.linspace(lat_min, lat_max, 925)
lons = np.linspace(lon_min, lon_max, 1667)
coords = [(round(x,4),round(y,4)) for x in lats for y in lons]

# Converting to a dataframe and adding a column for the grid square name
coords = pd.DataFrame(coords, columns=['lat_4dp', 'long_4dp'])
coords['grid_square'] = coords['lat_4dp'].map(str) + "," + coords['long_4dp'].map(str)
coords.head()

,lat_4dp,long_4dp,grid_square
0,51.257,-0.5420,"51.257,-0.542"
1,51.257,-0.5415,"51.257,-0.5415"
2,51.257,-0.5410,"51.257,-0.541"
3,51.257,-0.5405,"51.257,-0.5405"
4,51.257,-0.5400,"51.257,-0.54"


In [21]:
# Creating a list of all squares
all_squares = coords.grid_square
len(all_squares)

1541975

In [22]:
# Creating a list of danger squares
danger_squares = model3_dataset_danger.grid_square
len(danger_squares)

5000

In [71]:
# Creating a list of safe squares (all_squares minus danger_squares)
safe_squares = list(set(all_squares) - set(danger_squares))
print(len(safe_squares))
safe_squares = pd.DataFrame(safe_squares)
safe_squares.columns = ['grid_square']
safe_squares.head()

1536975


,grid_square
0,"51.2765,-0.4385"
1,"51.4545,-0.39"
2,"51.289,-0.287"
3,"51.4675,-0.485"
4,"51.3855,0.2395"


In [72]:
safe_squares_coords = safe_squares['grid_square'].str.split(',', expand=True)
safe_squares = pd.merge(safe_squares, safe_squares_coords, left_index=True, right_index=True)
safe_squares.columns = ['grid_square', 'latitude', 'longitude']
safe_squares.head()

,grid_square,latitude,longitude
0,"51.2765,-0.4385",51.2765,-0.4385
1,"51.4545,-0.39",51.4545,-0.39
2,"51.289,-0.287",51.289,-0.287
3,"51.4675,-0.485",51.4675,-0.485
4,"51.3855,0.2395",51.3855,0.2395


In [25]:
def myround(x, base=.0005):
    return base * round(x/base)

In [51]:
#lsoa_latlong['lat_4dp'] = myround(lsoa_latlong['Latitude'])
#lsoa_latlong['long_4dp'] = myround(lsoa_latlong['Longitude'])
lsoa_latlong.head()

,lsoa,Latitude,Longitude,lat_4dp,long_4dp
0,E01000001,51.520269,-0.0950,51.5205,-0.0950
1,E01000001,51.519848,-0.0967,51.5200,-0.0965
2,E01000001,51.519030,-0.0962,51.5190,-0.0960
3,E01000001,51.516904,-0.0981,51.5170,-0.0980
4,E01000003,51.522376,-0.0973,51.5225,-0.0975


### Adding traffic and population data to the safe squares

In [73]:
# Using cKDTree to add the LSOA names to the safe squares
# Extract lat and long subsets
square_coords = safe_squares[['latitude', 'longitude']]
lsoa_coords = lsoa_latlong[['Latitude', 'Longitude']]

# Construct tree
tree = cKDTree(lsoa_coords)

# Query the tree
distances, indices = tree.query(square_coords)

# Get the dataframe in the correct order
safe_squares_ordered = safe_squares.loc[indices]
safe_squares_ordered.reset_index(drop=True, inplace=True)

# Joining the dataframes
safe_squares = safe_squares_ordered.join(lsoa_latlong)

safe_squares.head()

,grid_square,latitude,longitude,lsoa,Latitude,Longitude,lat_4dp,long_4dp
0,"51.3945,-0.351",51.3945,-0.351,E01000001,51.520269,-0.0950,51.5205,-0.0950
1,"51.5045,0.2325",51.5045,0.2325,E01000001,51.519848,-0.0967,51.5200,-0.0965
2,"51.3945,-0.351",51.3945,-0.351,E01000001,51.519030,-0.0962,51.5190,-0.0960
3,"51.718,-0.361",51.718,-0.361,E01000001,51.516904,-0.0981,51.5170,-0.0980
4,"51.6305,-0.4075",51.6305,-0.4075,E01000003,51.522376,-0.0973,51.5225,-0.0975


In [93]:
safe_squares_ordered

,grid_square,latitude,longitude
0,"51.3945,-0.351",51.3945,-0.351
1,"51.5045,0.2325",51.5045,0.2325
2,"51.3945,-0.351",51.3945,-0.351
3,"51.718,-0.361",51.718,-0.361
4,"51.6305,-0.4075",51.6305,-0.4075
5,"51.3955,-0.459",51.3955,-0.459
6,"51.5045,-0.516",51.5045,-0.516
7,"51.576,-0.342",51.576,-0.342
8,"51.717,-0.1935",51.717,-0.1935
9,"51.649,0.2555",51.649,0.2555


In [74]:
distances.mean()

0.02819339665827583

In [85]:
safe_squares[['latitude', 'longitude']] = safe_squares[['latitude', 'longitude']].astype(float)

In [86]:
safe_squares.latitude.dtype

dtype('float64')

In [89]:
safe_squares = safe_squares.dropna()

In [91]:
len(safe_squares)

25053

In [92]:
def distance_m(lat1, lon1, lat2, lon2):
    """Returns the distance in metres between two points described by latitudes and longitudes using the Haversine formula"""
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return (12742 * asin(sqrt(a)))*1000
	
# Calculating the distances between the accident and traffic count locations in each row
distances_m = []                    
for index, row in safe_squares.iterrows():
    lat1 = row['latitude'] 
    lon1 = row['longitude'] 
    lat2 = row['Latitude'] 
    lon2 = row['Longitude'] 
    value = distance_m(lat1, lon1, lat2, lon2) 
    distances_m.append(value)
	
print("Mean:", round(np.mean(distances_m),1))
print("Median:", round(np.median(distances_m),1))
print("Minimum:", round(min(distances_m),1))
print("Maximum:", round(max(distances_m),1))

Mean: 24342.2
Median: 24125.1
Minimum: 292.6
Maximum: 60742.2


In [68]:
safe_squares_ordered.head()

,grid_square,latitude,longitude
0,"51.3945,-0.351",51.3945,-0.351
1,"51.5045,0.2325",51.5045,0.2325
2,"51.3945,-0.351",51.3945,-0.351
3,"51.718,-0.361",51.718,-0.361
4,"51.6305,-0.4075",51.6305,-0.4075


In [69]:
safe_squares_ordered.join(lsoa_latlong)

,grid_square,latitude,longitude,lsoa,Latitude,Longitude,lat_4dp,long_4dp
0,"51.3945,-0.351",51.3945,-0.351,E01000001,51.520269,-0.095000,51.5205,-0.0950
1,"51.5045,0.2325",51.5045,0.2325,E01000001,51.519848,-0.096700,51.5200,-0.0965
2,"51.3945,-0.351",51.3945,-0.351,E01000001,51.519030,-0.096200,51.5190,-0.0960
3,"51.718,-0.361",51.718,-0.361,E01000001,51.516904,-0.098100,51.5170,-0.0980
4,"51.6305,-0.4075",51.6305,-0.4075,E01000003,51.522376,-0.097300,51.5225,-0.0975
5,"51.3955,-0.459",51.3955,-0.459,E01000003,51.522085,-0.096000,51.5220,-0.0960
6,"51.5045,-0.516",51.5045,-0.516,E01000003,51.520924,-0.096500,51.5210,-0.0965
7,"51.576,-0.342",51.576,-0.342,E01000003,51.522014,-0.097200,51.5220,-0.0970
8,"51.717,-0.1935",51.717,-0.1935,E01000002,51.520970,-0.093800,51.5210,-0.0940
9,"51.649,0.2555",51.649,0.2555,E01000002,51.520504,-0.092800,51.5205,-0.0930


In [47]:
len(safe_squares_ordered)

1536975

In [46]:
len(safe_squares)

7812821

In [31]:
safe_squares.drop(['Latitude', 'Longitude'], axis=1, inplace=True)

In [32]:
safe_squares.lsoa.nunique()

4835

In [34]:
safe_squares.lsoa.value_counts(normalize=True, dropna=False)

NaN          0.983700
E01033583    0.000008
E01004749    0.000007
E01001646    0.000007
E01004252    0.000007
E01001756    0.000006
E01003475    0.000006
E01004012    0.000006
E01001859    0.000006
E01003482    0.000006
E01004745    0.000006
E01000718    0.000006
E01000092    0.000006
E01001754    0.000006
E01004656    0.000006
E01004142    0.000006
E01033725    0.000006
E01004215    0.000006
E01003617    0.000006
E01001519    0.000006
E01033724    0.000006
E01004509    0.000006
E01003367    0.000006
E01001748    0.000006
E01004222    0.000005
E01001731    0.000005
E01004393    0.000005
E01001948    0.000005
E01001437    0.000005
E01002684    0.000005
               ...   
E01033734    0.000002
E01033492    0.000002
E01033731    0.000002
E01002666    0.000002
E01033341    0.000002
E01002197    0.000002
E01032774    0.000002
E01000599    0.000002
E01033713    0.000002
E01033486    0.000002
E01001847    0.000002
E01001795    0.000002
E01002175    0.000002
E01003016    0.000002
E01002199 

In [56]:
safe_squares_merged = pd.merge(safe_squares, population, how='left', left_on='lsoa', right_on='LSOA')

In [60]:
len(population)

34753

In [30]:
# Using cKDTree to add the traffic to the safe squares
# Extract lat and long subsets
traffic_coords = traffic[['S Ref Latitude', 'S Ref Longitude']]
square_coords = safe_squares_merged[['latitude', 'longitude']]

# Construct tree
tree = cKDTree(traffic_coords)

# Query the tree
distances, indices = tree.query(square_coords)

# Get the dataframe in the correct order
traffic_ordered = traffic.loc[indices]
traffic_ordered.reset_index(drop=True, inplace=True)

# Joining the dataframes
safe_squares_merged = safe_squares_merged.join(traffic_ordered)

safe_squares_merged.head()

,grid_square,latitude,longitude,lsoa,LSOA,population_per_hectare,S Ref Latitude,S Ref Longitude,bicycle_aadf,motor_vehicle_aadf
0,"51.522,0.2015",51.522,0.2015,E01000001,E01000001,112.9,51.521151,0.203892,31.6,20922.2
1,"51.382,0.1175",51.382,0.1175,E01000001,E01000001,112.9,51.381924,0.106944,123.8,15964.6
2,"51.6855,-0.53",51.6855,-0.53,E01000001,E01000001,112.9,51.670716,-0.540878,43.4,13352.0
3,"51.652,0.009",51.652,0.009,E01000001,E01000001,112.9,51.666079,-0.000067,54.0,16842.6
4,"51.366,0.197",51.366,0.197,E01000003,E01000003,227.7,51.383861,0.211661,18.6,20441.6


In [32]:
safe_squares_merged.drop(['lsoa', 'LSOA', 'S Ref Latitude', 'S Ref Longitude'], axis=1, inplace=True)

In [33]:
safe_squares_merged.head()

,grid_square,latitude,longitude,population_per_hectare,bicycle_aadf,motor_vehicle_aadf
0,"51.522,0.2015",51.522,0.2015,112.9,31.6,20922.2
1,"51.382,0.1175",51.382,0.1175,112.9,123.8,15964.6
2,"51.6855,-0.53",51.6855,-0.53,112.9,43.4,13352.0
3,"51.652,0.009",51.652,0.009,112.9,54.0,16842.6
4,"51.366,0.197",51.366,0.197,227.7,18.6,20441.6


In [54]:
len(safe_squares_merged[safe_squares_merged['population_per_hectare'].isna()])

1511922

In [55]:
1536975 - 1511922

25053

### List of safe squares to sample

In [38]:
safe_squares_5000 = safe_squares_merged.sample(n=5000, replace=False, random_state=0)

In [39]:
safe_squares_5000.head()

,grid_square,latitude,longitude,population_per_hectare,bicycle_aadf,motor_vehicle_aadf
1045317,"51.3935,-0.2295",51.3935,-0.2295,NaN,64.4,5878.4
1229295,"51.412,-0.063",51.412,-0.063,NaN,110.0,11850.2
606106,"51.605,-0.524",51.605,-0.524,NaN,0.0,148872.0
4681,"51.4755,-0.398",51.4755,-0.398,99.8,408.2,53150.2
970575,"51.3205,-0.217",51.3205,-0.217,NaN,79.8,34077.8


In [40]:
safe_squares_5000.population_per_hectare.value_counts(normalize=True, dropna=False)

NaN       0.9842
 63.2     0.0002
 37.4     0.0002
 52.2     0.0002
 184.1    0.0002
 190.9    0.0002
 128.1    0.0002
 121.6    0.0002
 56.2     0.0002
 82.2     0.0002
 27.2     0.0002
 75.0     0.0002
 227.7    0.0002
 134.3    0.0002
 118.1    0.0002
 43.4     0.0002
 22.1     0.0002
 110.2    0.0002
 226.1    0.0002
 98.0     0.0002
 202.5    0.0002
 48.2     0.0002
 102.2    0.0002
 240.0    0.0002
 37.0     0.0002
 134.2    0.0002
 37.8     0.0002
 91.8     0.0002
 233.0    0.0002
 174.6    0.0002
           ...  
 27.4     0.0002
 145.6    0.0002
 125.4    0.0002
 118.2    0.0002
 55.5     0.0002
 284.8    0.0002
 141.3    0.0002
 149.8    0.0002
 45.4     0.0002
 39.6     0.0002
 68.1     0.0002
 235.0    0.0002
 150.8    0.0002
 164.0    0.0002
 71.2     0.0002
 79.9     0.0002
 43.9     0.0002
 99.8     0.0002
 103.7    0.0002
 22.9     0.0002
 7.4      0.0002
 88.1     0.0002
 232.4    0.0002
 53.4     0.0002
 15.9     0.0002
 58.5     0.0002
 55.1     0.0002
 140.2    0.00

In [36]:
safe_squares_download = list(safe_squares_5000.grid_square)

In [37]:
# Saving list to csv in case the kernel is restarted and the list is lost
# Will scrape images from this list in Scraping_Real_Images notebook
pd.Series(safe_squares_download).to_csv('data/accidents/safe_squares_download', index=False)

### Merging Safe and Danger datasets